# まとめ

* train (最初の 3 週間)
* valid A（第4週の前半）
* valid B（第4週の後半）
* all train (train + validA + validB)
* test (第5週の前半)
* LB (第5週の後半)


* 参考ベージ
https://www.kaggle.com/competitions/otto-recommender-system/discussion/370210

In [15]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [16]:
import pandas as pd

In [17]:
# TRAIN_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/input/otto-train-and-test-data-for-local-validation/'
# OUTPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/input/cris_baseline/output/'
TRAIN_DIR = '/kaggle/input/otto-train-and-test-data-for-local-validation/'

df_train = pd.read_parquet(TRAIN_DIR + 'train.parquet')
df_val = pd.read_parquet(TRAIN_DIR + 'test.parquet')

print(len(df_train))
print(len(df_val))

163955180
7683577


In [18]:
df_train.head()

,session,aid,ts,type
0,0,1517085,1659304800,0
1,0,1563459,1659304904,0
2,0,1309446,1659367439,0
3,0,16246,1659367719,0
4,0,1781822,1659367871,0


In [19]:
df_val.head()

,session,aid,ts,type
0,11098528,11830,1661119200,0
1,11098529,1105029,1661119200,0
2,11098530,264500,1661119200,0
3,11098530,264500,1661119288,0
4,11098530,409236,1661119369,0


# step1 sessionのごとにitem候補生成 : valid A 

covis matrix読み込み

In [20]:
%%time
VER = 6
DISK_PIECES = 4

def pqt_to_dict(df):
    return df.groupby('aid_x').aid_y.apply(list).to_dict()

# LOAD THREE CO-VISITATION MATRICES
# top_20_clicks = pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/kaggle/2022/OTTO/input/cris_baseline/output/top_20_clicks_v{VER}_0.pqt') )
# for k in range(1,DISK_PIECES): 
#     top_20_clicks.update( pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/kaggle/2022/OTTO/input/cris_baseline/output/top_20_clicks_v{VER}_{k}.pqt') ) )

top_20_clicks = pqt_to_dict( pd.read_parquet(f'/kaggle/input/compute-validation-score-cv-565/top_20_clicks_v{VER}_0.pqt') )
for k in range(1,DISK_PIECES): 
    top_20_clicks.update( pqt_to_dict( pd.read_parquet(f'/kaggle/input/compute-validation-score-cv-565/top_20_clicks_v{VER}_{k}.pqt') ) )

CPU times: user 43.1 s, sys: 3.38 s, total: 46.5 s
Wall time: 50.8 s


In [12]:
# step1
## 与えられたitemと1つのcovis_matrix
def generate_candidate(x, k=50):
    # 与えられたitem列をそのまま候補に
    candidates = x.aid.values.tolist()
    
    # 最後のitemからcovis_matrixを使って、候補を増やす
    last_item = candidates[-1]
    candidates.extend(top_20_clicks.get(last_item, []))
    
    # 最大50まで
    return candidates[:k]

In [10]:
from tqdm import tqdm
tqdm.pandas()

candidates = df_val.groupby('session').progress_apply(lambda x: generate_candidate(x)).explode()
candidates

100%|██████████| 1801251/1801251 [01:50<00:00, 16242.99it/s]


session
11098528      11830
11098528     588923
11098528    1732105
11098528     571762
11098528     884502
             ...   
12899778      13568
12899778    1097813
12899778    1102546
12899778    1068655
12899778    1495496
Length: 42573599, dtype: object

In [11]:
candidates.name = 'item'
candidates = candidates.to_frame().reset_index()
candidates

,session,item
0,11098528,11830
1,11098528,588923
2,11098528,1732105
3,11098528,571762
4,11098528,884502
...,...,...
42573594,12899778,13568
42573595,12899778,1097813
42573596,12899778,1102546
42573597,12899778,1068655


In [12]:
candidates.to_parquet('candidates.pqt')

# step2 item特徴 : train + valid A

In [13]:
%%time
item_features = pd.merge(df_train, df_val, on=['session','aid','type'], how='left').groupby('aid').agg({'aid': 'count', 'session': 'nunique', 'type': 'mean'})
item_features.columns = ['item_item_count', 'item_user_count', 'item_by_count']
item_features.to_parquet('item_features.pqt')
item_features

CPU times: user 2min 24s, sys: 19.2 s, total: 2min 43s
Wall time: 2min 43s


,item_item_count,item_user_count,item_by_count
aid,,,
0,33,26,0.000000
1,32,29,0.031250
2,12,12,0.000000
3,1201,668,0.079933
4,140,97,0.035714
...,...,...,...
1855598,6,6,0.000000
1855599,9,8,0.000000
1855600,68,42,0.073529


# step3 user特徴 : valid A

In [14]:
user_features = df_val.groupby('session').agg({'session': 'count', 'aid': 'nunique', 'type': 'mean'})
user_features.columns = ['user_user_count','user_item_count','user_buy_ratio']
# CONVERT COLUMNS TO INT32 and FLOAT32 HERE
user_features.to_parquet('user_features.pqt')
user_features

,user_user_count,user_item_count,user_buy_ratio
session,,,
11098528,1,1,0.000000
11098529,1,1,0.000000
11098530,6,2,0.166667
11098531,24,11,0.333333
11098532,2,2,0.000000
...,...,...,...
12899774,1,1,0.000000
12899775,1,1,0.000000
12899776,1,1,0.000000


# step4 : valid A の user × item 特徴量生成？

- user(session) x item(aid)に対するaction(type)の有無(1/0)を特徴量にする

In [13]:
def create_action_flag(df, type_number=0, column_name="user_item_click_flag"):
    action_flag_df = df.loc[df["type"] == type_number, ["session", "aid"]].drop_duplicates()
    action_flag_df[column_name] = 1
    return action_flag_df

In [16]:
user_item_click_flag = create_action_flag(df_val, type_number=0, column_name="user_item_click_flag")
user_item_cart_flag = create_action_flag(df_val, type_number=1, column_name="user_item_cart_flag")
user_item_order_flag = create_action_flag(df_val, type_number=2, column_name="user_item_order_flag")

In [17]:
tmp = pd.merge(
    user_item_click_flag,
    user_item_cart_flag,
    on=["session", "aid"],
    how="outer"
)
user_by_item_features = pd.merge(
    tmp,
    user_item_order_flag,
    on=["session", "aid"],
    how="outer"
).fillna(0)
user_by_item_features

,session,aid,user_item_click_flag,user_item_cart_flag,user_item_order_flag
0,11098528,11830,1.0,0.0,0.0
1,11098529,1105029,1.0,0.0,0.0
2,11098530,264500,1.0,0.0,0.0
3,11098530,409236,1.0,1.0,0.0
4,11098531,452188,1.0,0.0,1.0
...,...,...,...,...,...
5535985,12896465,1551275,0.0,0.0,1.0
5535986,12896465,1306971,0.0,0.0,1.0
5535987,12896768,1303029,0.0,0.0,1.0
5535988,12898765,73333,0.0,0.0,1.0


In [18]:
user_by_item_features.to_parquet('user_by_item_features.pqt')

# step5 : step1に step2,3,4を追加

In [37]:
candidates = pd.read_parquet('candidates.pqt')
candidates = candidates.rename(columns={'session':'user'})

item_features = pd.read_parquet('item_features.pqt')
candidates = candidates.merge(item_features, left_on='item', right_index=True, how='left').fillna(-1)
user_features = pd.read_parquet('user_features.pqt')
candidates = candidates.merge(user_features, left_on='user', right_index=True, how='left').fillna(-1)
user_features = pd.read_parquet('user_by_item_features.pqt')
candidates = candidates.merge(user_features, left_on='user', right_index=True, how='left').fillna(-1)
candidates

,user,item,item_item_count,item_user_count,item_by_count,user_user_count,user_item_count,user_buy_ratio,session,aid,user_item_click_flag,user_item_cart_flag,user_item_order_flag
0,11098528,11830,32546.0,18362.0,0.175659,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,11098528,588923,23956.0,14293.0,0.124937,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,11098528,1732105,10121.0,5565.0,0.280802,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,11098528,571762,18185.0,11918.0,0.125763,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,11098528,884502,30419.0,17385.0,0.135738,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42573594,12899778,13568,855.0,527.0,0.173099,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
42573595,12899778,1097813,462.0,328.0,0.151515,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
42573596,12899778,1102546,1054.0,877.0,0.097723,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
42573597,12899778,1068655,488.0,371.0,0.086066,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0


# step6 : 学習できる形にするgt:0,gt:1

In [38]:
tar = pd.read_parquet(TRAIN_DIR+'test_labels.parquet')
tar = tar.loc[ tar['type']=='clicks' ]
tar['item'] = tar['ground_truth'].explode().astype('int32')
tar['click'] = 1

tar = tar.rename(columns={'session':'user'})
tar

,user,type,ground_truth,item,click
0,11098528,clicks,[1679529],1679529,1
3,11098529,clicks,[1105029],1105029,1
6,11098532,clicks,[1596491],1596491,1
7,11098533,clicks,[1417450],1417450,1
10,11098534,clicks,[908024],908024,1
...,...,...,...,...,...
2212687,12899774,clicks,[1399483],1399483,1
2212688,12899775,clicks,[1760714],1760714,1
2212689,12899776,clicks,[1737908],1737908,1
2212690,12899777,clicks,[384045],384045,1


In [39]:
tar = tar[['user', 'item', 'click']]
tar

,user,item,click
0,11098528,1679529,1
3,11098529,1105029,1
6,11098532,1596491,1
7,11098533,1417450,1
10,11098534,908024,1
...,...,...,...
2212687,12899774,1399483,1
2212688,12899775,1760714,1
2212689,12899776,1737908,1
2212690,12899777,384045,1


In [40]:
candidates = candidates.merge(tar,on=['user','item'],how='left').fillna(0)

In [41]:
candidates

,user,item,item_item_count,item_user_count,item_by_count,user_user_count,user_item_count,user_buy_ratio,session,aid,user_item_click_flag,user_item_cart_flag,user_item_order_flag,click
0,11098528,11830,32546.0,18362.0,0.175659,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
1,11098528,588923,23956.0,14293.0,0.124937,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
2,11098528,1732105,10121.0,5565.0,0.280802,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
3,11098528,571762,18185.0,11918.0,0.125763,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
4,11098528,884502,30419.0,17385.0,0.135738,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42573594,12899778,13568,855.0,527.0,0.173099,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
42573595,12899778,1097813,462.0,328.0,0.151515,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
42573596,12899778,1102546,1054.0,877.0,0.097723,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
42573597,12899778,1068655,488.0,371.0,0.086066,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0


# Training

In [42]:
FEATURES = list(candidates.columns[2:8])
FEATURES

['item_item_count',
 'item_user_count',
 'item_by_count',
 'user_user_count',
 'user_item_count',
 'user_buy_ratio']

In [25]:
!pip install lightgbm

In [28]:
import lightgbm as lgb
from sklearn.model_selection import GroupKFold
import pickle

skf = GroupKFold(n_splits=5)
for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, candidates['click'], groups=candidates['user'] )):
    print(f'=====fold : {fold}=====')

    X_train = candidates.loc[train_idx, FEATURES]
    y_train = candidates.loc[train_idx, 'click']
    X_valid = candidates.loc[valid_idx, FEATURES]
    y_valid = candidates.loc[valid_idx, 'click']

    #参考　https://knuu.github.io/ltr_by_lightgbm.html

    params = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'lambdarank_truncation_level': 10,
        'ndcg_eval_at': [20],
        'n_estimators': 10000,
        'boosting_type': 'gbdt',
        'random_state': 0,
    }

    train = lgb.Dataset(X_train, y_train, group=candidates.loc[train_idx,:].groupby('user')['item'].agg('count').values )
    valid = lgb.Dataset(X_valid, y_valid, group=candidates.loc[valid_idx,:].groupby('user')['item'].agg('count').values )
     
     
    model = lgb.train(
        params, train, valid_sets=valid, 
        early_stopping_rounds=50, #50
        verbose_eval=5  # 10 round毎に metric を表示
    )
    
    #model.save_model(OUTPUT_DIR+f'chris_baseline_LGBM_fold{fold}_click.xgb')
    pickle.dump(model, open(f'lgb_model_{fold}.pkl','wb'))

=====fold : 0=====


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.595986 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1302
[LightGBM] [Info] Number of data points in the train set: 34058879, number of used features: 6
Training until validation scores don't improve for 50 rounds
[5]	valid_0's ndcg@20: 0.689515
[10]	valid_0's ndcg@20: 0.684408
[15]	valid_0's ndcg@20: 0.685152
[20]	valid_0's ndcg@20: 0.681689
[25]	valid_0's ndcg@20: 0.682378
[30]	valid_0's ndcg@20: 0.680798
[35]	valid_0's ndcg@20: 0.681289
[40]	valid_0's ndcg@20: 0.680226
[45]	valid_0's ndcg@20: 0.680933
[50]	valid_0's ndcg@20: 0.677506
Early stopping, best iteration is:
[1]	valid_0's ndcg@20: 0.698024
=====fold : 1=====
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.853434 seconds.
You can set `force_col_wise=true` to remove the overhead.
[

In [29]:
#メモリ節約
#del candidate, train, valid

# Inference

In [9]:
# TEST_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/input/otto-chunk-data-inparquet-format/'
TEST_DIR = '/kaggle/input/otto-chunk-data-inparquet-format/'

In [10]:
import glob

type_labels = {'clicks':0, 'carts':1, 'orders':2}

def load_test():    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(TEST_DIR + 'test_parquet/*')):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

df_test = load_test()
print('Test data has shape',df_test.shape)
df_test.head()

Test data has shape (6928123, 4)


,session,aid,ts,type
0,13099779,245308,1661795832,0
1,13099779,245308,1661795862,1
2,13099779,972319,1661795888,0
3,13099779,972319,1661795898,1
4,13099779,245308,1661795907,0


# 再step1 sessionのごとにitem候補生成 : test 

In [21]:
%%time

test_candidates = df_test.groupby('session').apply(generate_candidate).explode()
test_candidates

CPU times: user 1min 40s, sys: 1.61 s, total: 1min 41s
Wall time: 1min 42s


session
12899779      59625
12899779     941596
12899779     731692
12899779    1340695
12899779    1790770
             ...   
14571581     984794
14571581    1251890
14571581     990293
14571581    1072049
14571581    1764910
Length: 39331876, dtype: object

In [22]:
test_candidates.name = 'item'
test_candidates = test_candidates.to_frame().reset_index()
test_candidates

,session,item
0,12899779,59625
1,12899779,941596
2,12899779,731692
3,12899779,1340695
4,12899779,1790770
...,...,...
39331871,14571581,984794
39331872,14571581,1251890
39331873,14571581,990293
39331874,14571581,1072049


In [31]:
test_candidates.to_parquet('test_candidates.pqt')

# 再step2 item特徴 : all_train + test

In [23]:
%%time
all_train = pd.merge(df_train, df_val, on=['session','aid','type'], how='left')
all_item_features = pd.merge(all_train, df_test, on=['session','aid','type'], how='left').groupby('aid').agg({'aid': 'count', 'session': 'nunique', 'type': 'mean'})
all_item_features.columns = ['item_item_count', 'item_user_count', 'item_by_count']
all_item_features.to_parquet('all_item_features.pqt')
all_item_features

CPU times: user 4min 4s, sys: 31.2 s, total: 4min 35s
Wall time: 4min 36s


,item_item_count,item_user_count,item_by_count
aid,,,
0,33,26,0.000000
1,32,29,0.031250
2,12,12,0.000000
3,1201,668,0.079933
4,140,97,0.035714
...,...,...,...
1855598,6,6,0.000000
1855599,9,8,0.000000
1855600,68,42,0.073529


In [26]:
# del all_train
# gc.collect()

# 再step3 user特徴 :test

In [27]:
test_user_features = df_test.groupby('session').agg({'session': 'count', 'aid': 'nunique', 'type': 'mean'})
test_user_features.columns = ['user_user_count','user_item_count','user_buy_ratio']
# CONVERT COLUMNS TO INT32 and FLOAT32 HERE
test_user_features.to_parquet('test_user_features.pqt')
test_user_features

,user_user_count,user_item_count,user_buy_ratio
session,,,
12899779,1,1,0.000000
12899780,5,4,0.000000
12899781,11,5,0.090909
12899782,70,38,0.457143
12899783,11,9,0.000000
...,...,...,...
14571577,1,1,0.000000
14571578,1,1,0.000000
14571579,1,1,0.000000


# 再step4

In [28]:
test_user_item_click_flag = create_action_flag(df_test, type_number=0, column_name="user_item_click_flag")
test_user_item_cart_flag = create_action_flag(df_test, type_number=1, column_name="user_item_cart_flag")
test_user_item_order_flag = create_action_flag(df_test, type_number=2, column_name="user_item_order_flag")

In [29]:
tmp = pd.merge(
    test_user_item_click_flag,
    test_user_item_cart_flag,
    on=["session", "aid"],
    how="outer"
)
test_user_by_item_features = pd.merge(
    tmp,
    test_user_item_order_flag,
    on=["session", "aid"],
    how="outer"
).fillna(0)
test_user_by_item_features

,session,aid,user_item_click_flag,user_item_cart_flag,user_item_order_flag
0,13099779,245308,1.0,1.0,0.0
1,13099779,972319,1.0,1.0,0.0
2,13099780,354638,1.0,0.0,0.0
3,13099781,669617,1.0,0.0,0.0
4,13099781,1055072,1.0,0.0,0.0
...,...,...,...,...,...
5006552,13997922,709932,0.0,0.0,1.0
5006553,13998939,1018649,0.0,0.0,1.0
5006554,13998939,751788,0.0,0.0,1.0
5006555,13999077,750729,0.0,0.0,1.0


In [30]:
test_user_by_item_features.to_parquet('test_user_by_item_features.pqt')

# 再step5 : 再step1に 再step2,3を追加

In [32]:
test_candidates = pd.read_parquet("test_candidates.pqt")
test_candidates = test_candidates.rename(columns={'session':'user'})

test_item_features = pd.read_parquet('all_item_features.pqt')
test_candidates = test_candidates.merge(test_item_features, left_on='item', right_index=True, how='left').fillna(-1)
test_user_features = pd.read_parquet('test_user_features.pqt')
test_candidates = test_candidates.merge(test_user_features, left_on='user', right_index=True, how='left').fillna(-1)
test_user_by_item_features = pd.read_parquet('test_user_by_item_features.pqt')
test_candidates = test_candidates.merge(test_user_by_item_features, left_on='user', right_index=True, how='left').fillna(-1)

test_candidates

,user,item,item_item_count,item_user_count,item_by_count,user_user_count,user_item_count,user_buy_ratio,session,aid,user_item_click_flag,user_item_cart_flag,user_item_order_flag
0,12899779,59625,10.0,9.0,0.000000,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,12899779,941596,68.0,44.0,0.073529,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,12899779,731692,85.0,42.0,0.188235,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,12899779,1340695,11.0,8.0,0.090909,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,12899779,1790770,30.0,20.0,0.133333,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39331871,14571581,984794,846.0,623.0,0.054374,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
39331872,14571581,1251890,21958.0,14224.0,0.050824,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
39331873,14571581,990293,3958.0,2533.0,0.077312,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
39331874,14571581,1072049,503.0,295.0,0.174950,1,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [33]:
# test_candidates

# pred

In [43]:
import pickle
import numpy as np

preds = np.zeros(len(test_candidates))
for fold in range(5):
    with open(f'lgb_model_{fold}.pkl', 'rb') as f:
        model = pickle.load(f)
    preds += model.predict(test_candidates[FEATURES], num_iteration=model.best_iteration)/5

predictions = test_candidates[['user','item']].copy()
predictions['pred'] = preds
predictions

,user,item,pred
0,12899779,59625,0.036153
1,12899779,941596,0.021093
2,12899779,731692,0.021093
3,12899779,1340695,0.036153
4,12899779,1790770,0.021093
...,...,...,...
39331871,14571581,984794,-0.011851
39331872,14571581,1251890,-0.026433
39331873,14571581,990293,-0.013169
39331874,14571581,1072049,-0.004840


In [44]:
predictions = predictions.sort_values(['user','pred'], ascending=[True,False]).reset_index(drop=True)
predictions

,user,item,pred
0,12899779,59625,0.036153
1,12899779,1340695,0.036153
2,12899779,637538,0.036153
3,12899779,985764,0.036153
4,12899779,110837,0.036153
...,...,...,...
39331871,14571581,636390,-0.013169
39331872,14571581,1791780,-0.013169
39331873,14571581,775327,-0.013169
39331874,14571581,990293,-0.013169


In [45]:
predictions['n'] = predictions.groupby('user').item.cumcount().astype('int8')
predictions

,user,item,pred,n
0,12899779,59625,0.036153,0
1,12899779,1340695,0.036153,1
2,12899779,637538,0.036153,2
3,12899779,985764,0.036153,3
4,12899779,110837,0.036153,4
...,...,...,...,...
39331871,14571581,636390,-0.013169,16
39331872,14571581,1791780,-0.013169,17
39331873,14571581,775327,-0.013169,18
39331874,14571581,990293,-0.013169,19


In [46]:
predictions = predictions.loc[predictions.n<20]
predictions

,user,item,pred,n
0,12899779,59625,0.036153,0
1,12899779,1340695,0.036153,1
2,12899779,637538,0.036153,2
3,12899779,985764,0.036153,3
4,12899779,110837,0.036153,4
...,...,...,...,...
39331870,14571581,1158237,-0.013169,15
39331871,14571581,636390,-0.013169,16
39331872,14571581,1791780,-0.013169,17
39331873,14571581,775327,-0.013169,18


In [47]:
sub = predictions.groupby('user').item.apply(list)
sub = sub.to_frame().reset_index()
sub

,user,item
0,12899779,"[59625, 1340695, 637538, 985764, 110837, 42681..."
1,12899780,"[1502122, 636101, 582732, 736515, 1360606, 103..."
2,12899781,"[199008, 199008, 199008, 199008, 199008, 19900..."
3,12899782,"[1494780, 1494780, 1494780, 1494780, 779477, 7..."
4,12899783,"[1830736, 1218, 1817895, 1811433, 58861, 19838..."
...,...,...
1671798,14571577,"[1141710, 1289343, 367734, 86916, 242737, 5881..."
1671799,14571578,"[1455220, 519105, 815460, 5573, 822641, 181171..."
1671800,14571579,"[702275, 898561, 870569, 739876, 1550479, 8579..."
1671801,14571580,"[1627186, 891417, 679257, 202353, 1314576, 888..."


In [48]:
sub.item = sub.item.apply(lambda x: " ".join(map(str,x)))
sub

,user,item
0,12899779,59625 1340695 637538 985764 110837 426813 9415...
1,12899780,1502122 636101 582732 736515 1360606 1032776 6...
2,12899781,199008 199008 199008 199008 199008 199008 1990...
3,12899782,1494780 1494780 1494780 1494780 779477 779477 ...
4,12899783,1830736 1218 1817895 1811433 58861 198385 1729...
...,...,...
1671798,14571577,1141710 1289343 367734 86916 242737 5881 84255...
1671799,14571578,1455220 519105 815460 5573 822641 1811714 5380...
1671800,14571579,702275 898561 870569 739876 1550479 857928 143...
1671801,14571580,1627186 891417 679257 202353 1314576 888228 12...


In [49]:
sub.columns = ['session_type','labels']
sub.session_type = sub.session_type.astype('str')+ '_clicks'
sub

,session_type,labels
0,12899779_clicks,59625 1340695 637538 985764 110837 426813 9415...
1,12899780_clicks,1502122 636101 582732 736515 1360606 1032776 6...
2,12899781_clicks,199008 199008 199008 199008 199008 199008 1990...
3,12899782_clicks,1494780 1494780 1494780 1494780 779477 779477 ...
4,12899783_clicks,1830736 1218 1817895 1811433 58861 198385 1729...
...,...,...
1671798,14571577_clicks,1141710 1289343 367734 86916 242737 5881 84255...
1671799,14571578_clicks,1455220 519105 815460 5573 822641 1811714 5380...
1671800,14571579_clicks,702275 898561 870569 739876 1550479 857928 143...
1671801,14571580_clicks,1627186 891417 679257 202353 1314576 888228 12...


In [51]:
sub.to_csv("submission.csv", index=False)

# save

In [50]:
with open('chris_reranker_baseline_click_preds.pkl','wb') as f:
    pickle.dump(sub, f)